In [1]:
### https://anaconda.org/anaconda/py-xgboost
### Importando as libs
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import confusion_matrix, mean_squared_error
import pylab as pl
from sklearn import model_selection
from itertools import combinations

# Coleta de dados

In [2]:
def forward_feature_construction(colunasx,colunasy):
    ### Config
    seed = 9 
    teste_size=0.30 ##30%
    size = len(colunasx)
    lista = list()
    for id in  range(1,size):
        for x in  combinations(colunasx,r=id):
            lista.append(list(x))
        ### Resampling
    return lista    

## Fontes

In [3]:
### Coletando as fontes de dados
test_csv = pd.read_csv('r_teste.csv', sep = ',', encoding = 'UTF-8',header='infer')
train_csv = pd.read_csv('r_treino.csv', sep = ',', encoding = 'UTF-8',header='infer')
train_dropna_csv = pd.read_csv('r_treino_dropna.csv', sep = ',', encoding = 'UTF-8',header='infer')

In [4]:
colunasx=[
    'Classe',
    'FamiliaQtde',
    'PassagemPreco',
    'SexoNum',
    'SaudacaoNum'   
]
colunasy=['Sobreviventes']

In [5]:
print(forward_feature_construction(colunasx,colunasy))

[['Classe'], ['FamiliaQtde'], ['PassagemPreco'], ['SexoNum'], ['SaudacaoNum'], ['Classe', 'FamiliaQtde'], ['Classe', 'PassagemPreco'], ['Classe', 'SexoNum'], ['Classe', 'SaudacaoNum'], ['FamiliaQtde', 'PassagemPreco'], ['FamiliaQtde', 'SexoNum'], ['FamiliaQtde', 'SaudacaoNum'], ['PassagemPreco', 'SexoNum'], ['PassagemPreco', 'SaudacaoNum'], ['SexoNum', 'SaudacaoNum'], ['Classe', 'FamiliaQtde', 'PassagemPreco'], ['Classe', 'FamiliaQtde', 'SexoNum'], ['Classe', 'FamiliaQtde', 'SaudacaoNum'], ['Classe', 'PassagemPreco', 'SexoNum'], ['Classe', 'PassagemPreco', 'SaudacaoNum'], ['Classe', 'SexoNum', 'SaudacaoNum'], ['FamiliaQtde', 'PassagemPreco', 'SexoNum'], ['FamiliaQtde', 'PassagemPreco', 'SaudacaoNum'], ['FamiliaQtde', 'SexoNum', 'SaudacaoNum'], ['PassagemPreco', 'SexoNum', 'SaudacaoNum'], ['Classe', 'FamiliaQtde', 'PassagemPreco', 'SexoNum'], ['Classe', 'FamiliaQtde', 'PassagemPreco', 'SaudacaoNum'], ['Classe', 'FamiliaQtde', 'SexoNum', 'SaudacaoNum'], ['Classe', 'PassagemPreco', 'SexoN

In [6]:
### Config
seed = 9 
teste_size=0.30 ##30%

In [7]:
train_dropna_csv=train_dropna_csv.set_index(train_dropna_csv['PassageiroId'])

In [8]:
x_treino_dropna, x_teste_dropna, y_treino_dropna, y_teste_dropna = model_selection.train_test_split(train_dropna_csv[colunasx],train_dropna_csv[colunasy],test_size=teste_size,random_state=seed)

## Resampling

## Modelo - Treino

In [9]:
xgb_model = xgb.XGBClassifier(max_depth=3).fit(x_treino_dropna, y_treino_dropna)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## Modelo - Predict

In [10]:
predictions = xgb_model.predict(x_teste_dropna)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [11]:
actuals = y_teste_dropna
print(confusion_matrix(actuals, predictions))

[[120   9]
 [ 25  61]]


In [12]:
previsto = pd.DataFrame({'previsto':predictions},index=actuals.index)
resultadoErros =       previsto.loc[previsto['previsto'].values != actuals['Sobreviventes'].values]
resultadoAcertos =     previsto.loc[previsto['previsto'].values == actuals['Sobreviventes'].values]

In [13]:
resultXGBClassifier = xgb_model.score(x_teste_dropna,y_teste_dropna)
print('Score XGBClassifier:',(resultXGBClassifier*100.0))

Score XGBClassifier: 84.18604651162791


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [14]:
print('Predict',x_teste_dropna.shape)
print('Erros',resultadoErros.shape)
print('Acertos',resultadoAcertos.shape)

Predict (215, 5)
Erros (34, 1)
Acertos (181, 1)


# Resultado Final

In [15]:
y_pred_dropna = xgb_model.predict(test_csv[colunasx])

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [ ]:
submission = pd.DataFrame({
        "PassengerId": test_csv["PassageiroId"],
        "Survived": y_pred_dropna
    })
submission.to_csv('resultXGBClassifier_dropna.csv', index=False)

In [ ]:
xgb.plot_tree(xgb_model)
fig = pl.gcf()
fig.set_size_inches(150, 100)